# neoPZ Documentation

## Table of contents

1. [Topology](#topology)
2. [Geometry](#geometry)

<a id="topology"></a>
## Topology

The topology of all elements implemented on NeoPZ are grouped in pztopology. The classes associated to this namespace are listed in the following:

 
1. TPZPoint
2. TPZLine
3. TPZQuadrilateral
4. TPZTriangle
5. TPZCube
6. TPZPrism
7. TPZPyramid
8. TPZTetrahedron

Each element (e.g. point, line, quadrilateral, triangle, etc) is considered as the union of its sides. Each side is composed of an open set of points ans has inherent dimension and topology.

There is a unique side whose closure is the proper element. By convention this is the highest numbered side.

To each element a parameter space can be associated which is commonly known as the master element space.

### Main atributions of the class Topology

#### a. Definition of number of corners, sides, dimension and faces.

Side are open sets of points. Within NeoPZ sides are associated with typical finite element topologies: point, line, quadrilateral, triangle, tetrahedra, hexahedra, prism, pyramid.

An element is a closed set of points. Each element can be partioned into its sides (an intersection os its sides is empty and their union is the element).

Zero dimensional sides are called corners. A corner is both open and closed set.

In [1]:
// Make sure to change '/usr/local/' according to the installation in your computer
#pragma cling add_include_path("/usr/local/pzlib/include")
#pragma cling add_include_path("/usr/local/pzlib/include/Topology")
#pragma cling add_include_path("/usr/local/pzlib/include/Matrix")
#pragma cling add_include_path("/usr/local/pzlib/include/Util")
#pragma cling add_include_path("/usr/local/pzlib/include/Common")
#pragma cling add_include_path("/usr/local/pzlib/include/Save")
#pragma cling add_include_path("/usr/local/pzlib/include/Integral")
#pragma cling add_include_path("/usr/local/pzlib/include/LinearSolver")
#pragma cling add_include_path("/usr/local/pzlib/include/PerfUtil")

#pragma cling add_library_path("/usr/local/lib/")

//#pragma cling load("libpz.so")


#include <iostream>

#include "pzvec.h"
//#include "tpzintpoints.h"
//#include "pzquad.h"
#include "tpzquadrilateral.h"

TPZVec<REAL> blob;
blob.resize(3);
blob[0] = 10.;
std::cout << blob[0] << std::endl;

// Link error to be solved:
pztopology::TPZQuadrilateral quad;
std::cout << quad.NCornerNodes << std::endl;
std::cout << quad.NSides << std::endl;
std::cout << quad.Dimension << std::endl;
std::cout << quad.NFaces << std::endl;

SyntaxError: invalid syntax (<ipython-input-1-73ee319dbeb5>, line 1)

#### b. Definition of the dimension of each side and associated corner nodes.

Each side has an associated topological dimension and a given number of corner nodes:

|    Topology   | Dimension | NCornerNodes |
|:-------------:|:---------:|:------------:|
|     Point     |     0     |       1      |
|      Line     |     1     |       2      |
|    Triangle   |     2     |       3      |
| Quadrilateral |     2     |       4      |
|   Tetrahedra  |     3     |       4      |
|   Hexahedra   |     3     |       8      |
|     Prism     |     3     |       6      |
|    Pyramid    |     3     |       5      |

In [2]:
int side = 2;
int dimside = quad.SideDimension(side);
std::cout << "The dimension of the side " << side << " is" << dimside << std::endl;
int nsidenodes = quad.NSideNodes(side);
std::cout << "The number of associated corner nodes of the side " << side << " is " << nsidenodes << std::endl;

SyntaxError: invalid syntax (<ipython-input-2-244d53a64d60>, line 1)

#### c. Definition of the parametric transformation between sides 

Each element is partitioned by its sides. The closure of a side includes a number os sides and forms an element topology. This imples that each point on the boundary of a side belongs to a side of lower dimension.

A point in the parametric domain of a side can be transformed to a point in the parametric domain of the element. The transformation is a homogeneous transformation (i.e. a linear transformation followed by a translation).

These functionalities are implemented by the method *SideToSideTransform*. It is assumed that *sidefrom* belongs to *sideto*. If *sidefrom* corresponds to the volume of the element, the method will return a transformation that is a projection of the interior point to the side.

In [ ]:
# int sidefrom = 2;
# int sideto = 1;
# TPZTransform<> tr = quad.SideToSideTransform(sidefrom, sideto);

#### d. Creation of integration rules associated to each side

As each side is associated with an element topology, a specific integration rule exists for each side of an element. The integration rule classes associated with each element topology are:

|  **Topology** | **Dimension** | **Class name of integration rule** |
|:-------------:|:-------------:|:----------------------------------:|
|    Abstract   |               |            TPZIntPoints            |
|     Point     |       0       |            TPZInt1Point            |
|      Line     |       1       |              TPZInt1d              |
|    Triangle   |       2       |           TPZIntTriangle           |
| Quadrilateral |       2       |             TPZIntQuad             |
|   Tetrahedra  |       3       |            TPZIntTetra3D           |
|   Hexahedra   |       3       |            TPZIntCube3D            |
|     Prism     |       3       |            TPZIntPrism3D           |
|    Pyramid    |       3       |           TPZIntPyramid3D          |

The method responsible to create the integration rule associated with a side is *CreateSideIntegrationRule*. The input of this method are the *side* and the *order* of the integration rule to be created. The output is a pointer to *TPZIntPoints* which corresponds to the created integration rule.


In [ ]:
# int side = 0;
# int integr_order = 3;
# TPZIntPoints * integr = quad.CreateSideIntegrationRule(side, integr_order)

#### e. Definition of a transformation index associated with a side.

When working with high order hierarchical shape functions the orientation of the side will influence the values of the functions associated with the side. In order to make the functions unique an orientation is associated with each side which depends on an identifier associated with each node. The transformations associated with lines, triangles and quadrilaterals are documented in [NeoPZ shape functions](https://www.sciencedirect.com/science/article/pii/S0045782509000255?via%3Dihub).

The method that computes the transformation index is *int GetTransformId(TPZVec<int64_t> &id)*. The input is a vector with the indexes associated with the corner nodes of a side and the output is its the transformation index.

In [ ]:
# TPZVec<int64_t> id;
# int transf_id = quad.GetTransformId(id);

#### f. Definition of the permutation index associated with the element.



In [ ]:
# TPZVec<int> permgather; // output variable
# quad.GetSideHDivPermutation(transf_id, permgather);

#### g. Relationship between sides: which sides are included in the closure of a side and orientation (note that the output variable is a stack object).

In [ ]:
# TPZStack<int> high;
# quad.HigherDimensionSides(side, high);

#### h. Projection of a point to a side (this function is actually implemented on TPZShape).

In [ ]:
# int rib;
# TPZVec<REAL> in;
# REAL out;
# pzshape::TPZShapeQuad::ProjectPoint2dQuadToRib(rib, in, out);

<a id="geometry"></a>
## Geometry

The geometry of all elements types implemented on NeoPZ are grouped in pzgeom. The classes associated to this namespace are listed in the following:

 
1. TPZGeoPoint
2. TPZGeoLinear
3. TPZGeoQuad
4. TPZGeoTriangle
5. TPZGeoCube
6. TPZGeoPrism
7. TPZGeoPyramid
8. TPZGeoTetrahedra

Geometry templates implement the mapping between the master element and actual element. The geometry template also stores the indices of the corner nodes.
The primary attribute of the geometric template is to compute the coordinate as a function of the position in master element coordinate (X) and its gradient (GradX). The standard way of computing X and GradX is by using linear interpolation. As such, the geometric template class implements the linear shape functions associated with its topology.